## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* -0.24
* -0.04  
* **0.07**
* 0.27


In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
user_question = "I just discovered the course. Can I still join it?"
encoded_question = embedding_model.encode(user_question)
encoded_question[0]

0.078222655

In [10]:
# The model encoding dim
len(encoded_question)

768

Loading documents with unique ids  
We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
documents[:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'}]

In [8]:
len(documents)

948

In [9]:
import json

# Function to filter data based on the 'course' attribute
def filter_by_course(data, course_name):
    return [item for item in data if item.get('course') == course_name]

# Filter data where course is 'Math'
filtered_data = filter_by_course(documents, 'machine-learning-zoomcamp')

# Output the filtered data
print(len(filtered_data))

375


## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`


In [55]:
embeddings = []

for doc in filtered_data :
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    doc["qt_vector"] = embedding_model.encode(qa_text).tolist()
    embeddings.append(doc["qt_vector"])

In [56]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

In [15]:
v = encoded_question
scores = X.dot(v)
scores.max()

0.6506573240979582

### vector search

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_data, embeddings=X)
search_engine.search(v, num_results=5)


## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

What did you get?

* **0.93**
* 0.73
* 0.53
* 0.33

In [18]:
#First, load the ground truth dataset:

import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')


In [20]:
df_ground_truth.head(5)

,question,course,document
2123,Where can I sign up for the course?,machine-learning-zoomcamp,0227b872
2124,Can you provide a link to sign up?,machine-learning-zoomcamp,0227b872
2125,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0227b872
2126,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0227b872
2127,How can I structure my questions and answers f...,machine-learning-zoomcamp,0227b872


In [19]:
# hit rate
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [51]:
def question_vector(question):
    v_q = embedding_model.encode(question)
    return search_engine.search(v_q, num_results=5)

In [52]:
from tqdm.auto import tqdm

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_vector(q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:25<00:00, 21.34it/s]


In [53]:
hit_rate(relevance_total)

0.9398907103825137

## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?


In [62]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 


In [71]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qt_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [72]:
for doc in tqdm(filtered_data):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:12<00:00, 29.73it/s]


In [76]:
def elastic_search_knn(field, vector):
    
    knn_query = {
    "field": "qt_vector",
    "query_vector": vector,
    "k": 5,
    "num_candidates": 10000
    }

    search_query = {
        "knn": knn_query,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [74]:
def question_text_vector_knn(question):
    v_q = embedding_model.encode(question)
    return elastic_search_knn('question_text_vector', v_q)

In [81]:
question_text_vector_knn(user_question)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

Highest score id:  ee58a693

In [77]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = question_text_vector_knn(q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:26<00:00, 21.17it/s]


In [78]:
hit_rate(relevance_total)

0.9398907103825137